<a href="https://colab.research.google.com/github/AlineNascimento/GEE_codes/blob/main/Take_data_from_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Take Data from GEE
# Criado por: Aline Andrade do Nascimento
# alinephysics@gmail.com

#Importação da bibliotecas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install pandas fiona shapely pyproj rtree

In [ ]:
pip install geopandas

In [ ]:
pip install netCDF4

In [ ]:
# Libraries to open 'netcdf' file
import netCDF4 as netcdf
import xarray

# To pre-process dataframe and use in visualization
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point
import ast

#  Conectando o GEE com outros serviços do Google
A API do Python do Earth Engine e as ferramentas de linha de comando podem ser instaladas usando a ferramenta de instalação do pacote pip do Python.

In [ ]:
!pip install earthengine-api

In [ ]:

!earthengine authenticate

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import the Earth Engine API and initialize it.
import ee
ee.Initialize()

In [ ]:
import folium
token = '4/1AX4XfWif2ASmQd28DqQmuR9blSSDRQqVHJ8d3Jri9TIPA4h832W7H1dWeKY'
# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

In [ ]:
pip install geemap

In [ ]:
import json
import geemap
import geemap.colormaps as cm
import altair as alt  ##biblioteca de visualização(gráficos)
import folium

In [ ]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

# 1. Visão geral do redutor

Os redutores são a maneira de agregar dados ao longo do tempo, espaço, bandas, matrizes e outras estruturas de dados no Earth Engine. A classe `ee.Reducer` especifica como os dados são agregados. Os redutores nesta classe podem especificar uma estatística simples a ser usada para a agregação (por exemplo, mínimo, máximo, média, mediana, desvio padrão etc.), ou um resumo mais complexo dos dados de entrada (por exemplo, histograma, regressão linear, lista) . As reduções podem ocorrer ao longo de:

- **time**  = `imageCollection.reduce()`
- **space** = `image.reduceRegion()` and `image.reduceNeighborhood()`**
- **bands**  = `image.reduce()`,

Ou o espaço de atributo de um `FeatureCollection` (`featureCollection.reduceColumns()` ou `FeatureCollection` métodos que começam com`aggregate_`).

## Os redutores possuem entradas e saídas

Os redutores pegam um conjunto de dados de entrada e produzem uma única saída. Quando um único redutor de entrada é aplicado a uma imagem multibanda, o Earth Engine replica automaticamente o redutor e o aplica separadamente a cada banda. Como resultado, a imagem de saída tem o mesmo número de bandas que a imagem de entrada; cada banda na saída é a redução de pixels da banda correspondente nos dados de entrada. Alguns redutores usam tuplas de conjuntos de dados de entrada. Esses redutores não serão replicados automaticamente para cada banda. Por exemplo, `ee.Reducer.LinearRegression()` leva vários conjuntos de dados de previsão (representando variáveis ​​independentes na regressão) em uma ordem específica (consulte a seção Redutores de regressão abaixo).

Alguns redutores produzem várias saídas, por exemplo `ee.Reducer.minMax()`, `ee.Reducer.histogram()` or `ee.Reducer.toList()`. For example:

In [ ]:
geometry = ee.FeatureCollection("projects/ee-alinephysics/assets/mapbiomas/RegiaoAmazonia")

In [ ]:
road = geometry.limit(50)
centroid = road.geometry().centroid().getInfo()['coordinates']
centroid.reverse()
Mapdisplay(centroid,{'road':road.getInfo()},zoom_start=5)

In [ ]:
# Load and filter the Sentinel-2 image collection.
collection = ee.Image('projects/mapbiomas-workspace/public/collection6/mapbiomas_collection60_integration_v1')

# Reduce the collection.
extrema = collection.reduce(ee.Reducer.mean())

In [ ]:
geometry.getInfo()

In [ ]:
center =  [-2.145746, -59.004845]
vizParams = {
    'bands': 'classification_2019'
        }

dicc =  {'mapbiomas mosaic': collection.getMapId(vizParams)}

Mapdisplay(center,dicc,zoom_start=4)

Isso produzirá uma saída com o dobro do número de bandas das entradas, onde os nomes das bandas na saída têm '_min' ou '_max' anexados ao nome da banda.

 O tipo de saída deve corresponder ao cálculo. Por exemplo, um redutor aplicado a uma ImageCollection tem uma saída de imagem. Como a saída é interpretada como um valor de pixel, você deve usar redutores com uma saída numérica para reduzir um ImageCollection (reducers like `toList()` or `histogram()` won’t work).

## Redutores usam entradas ponderadas

Por padrão, as reduções sobre os valores de pixel são ponderadas por sua máscara, embora esse comportamento possa ser alterado (consulte a seção Ponderação abaixo). Pixels com máscara igual a 0 não serão utilizados na redução.

## Combinando redutores

Se sua intenção é aplicar vários redutores às mesmas entradas, é uma boa prática `combine()` os redutores para eficiência. Especificamente, chamar `combine()` em um redutor com `sharedInputs` definido como `true` resultará em apenas uma única passagem pelos dados. Por exemplo, para calcular a média e o desvio padrão dos pixels em uma imagem, você pode usar algo assim:

In [ ]:
from pprint import pprint

# Load a Landsat 8 image.
image = ee.Image('projects/mapbiomas-workspace/public/collection6/mapbiomas_collection60_integration_v1')

# Combine the mean and standard deviation reducers.
reducers = ee.Reducer.median()

# Use the combined reducer to get the mean and SD of the image.
stats = image.reduceRegion(
  reducer=reducers,
  bestEffort=True,
)

# Display the dictionary of band means and SDs.
pprint(stats.getInfo())

Na saída, observe que os nomes dos redutores foram anexados aos nomes das entradas para distinguir as saídas do redutor. Esse comportamento também se aplica às saídas de imagem, que terão o nome do redutor anexado aos nomes das bandas de saída.

# 2. Reduções na coleção de imagens

Considere o exemplo da necessidade de obter a mediana sobre uma série temporal de imagens representadas por uma `ImageCollection`. Para reduzir um `ImageCollection`, use `imageCollection.reduce()`. Isso reduz a coleção de imagens a uma imagem individual, conforme ilustrado na Figura 1. Especificamente, a saída é calculada pixel a pixel, de modo que cada pixel na saída seja composto pelo valor médio de todas as imagens da coleção naquele local. Para obter outras estatísticas, como média, soma, variância, um percentil arbitrário, etc., o redutor apropriado deve ser selecionado e aplicado. Para estatísticas básicas como min, max, mean, etc., `ImageCollection` tem métodos de atalho como `min()`, `max()`, `mean()`, etc. Eles funcionam exatamente da mesma maneira que chamar reduce (), exceto os nomes de bandas resultantes não terão o nome do redutor anexado.


<center>
<image src = "https://developers.google.com/earth-engine/images/Reduce_ImageCollection.png">  
</center>

<center>
  Figure 1. Illustration of an ee.Reducer applied to an ImageCollection.

In [ ]:
# Load an image collection, filtered so it's not too much data.
collection = ee.ImageCollection('LANDSAT/LT05/C01/T1')\
               .filterDate('2008-01-01', '2008-12-31')\
               .filter(ee.Filter.eq('WRS_PATH', 44))\
               .filter(ee.Filter.eq('WRS_ROW', 34))

# Compute the median in each band, each pixel.
# Band names are B1_median, B2_median, etc.
median = collection.reduce(ee.Reducer.median())

# The output is an Image.  Add it to the map.
vis_param = {'bands': ['B4_median', 'B3_median', 'B2_median'], 'gamma': 1.6}
median_tk = median.getMapId(vis_param)

center = [37.7924, -122.3355]
Mapdisplay(center,{'Landsat 5':median_tk},zoom_start=9)

This returns a multi-band Image, each pixel of which is the median of all unmasked pixels in the ImageCollection at that pixel location. Specifically, the reducer has been repeated for each band of the input imagery. Note that the band names have the name of the reducer appended: ‘B1_median’, ‘B2_median’, etc.


# 3. Reduções de imagem

Para reduzir uma `Image`, use `image.reduce()`. Reduzir uma imagem funciona de maneira análoga a `imageCollection.reduce()`, exceto que as bandas da imagem são inseridas no redutor em vez das imagens na coleção. A saída também é uma imagem com número de bandas igual ao número de saídas do redutor. Por exemplo:




In [ ]:
import time

In [ ]:
image = ee.Image('projects/mapbiomas-workspace/public/collection6/mapbiomas_collection60_integration_v1')\
          .select(['classification_2019'])

# Reduce the image to get a one-band maximum value image.
clipValue = image.clip(gm)

# Display the result
vis_param = {'max': 13000, 'gamma': 1.6}
center =  [-2.145746, -59.004845]

clipValue_tk = clipValue.getMapId()
Mapdisplay(center,{'Landsat 5':clipValue_tk},zoom_start=4)

# Export.table.toDrive({
#     collection: reduced,
#     description: 'Mapbiomas',
#     folder: 'Mapbiomas',
#     fileNamePrefix: 'Mapbiomas-DadosOCO2-20-31082019',
#     fileFormat: 'CSV'
# })

# Export the FeatureCollection to a KML file.
task = ee.batch.Export.table.toDrive(**{
  'collection': clipValue,
  'folder': "TESTE_HOJE",
  'description':'vectorsToDriveExample',
  'fileNamePrefix': 'Mapbiomas-DadosOCO2-20-31082019',
  'fileFormat': 'CSV'
})
task.start()

while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

In [ ]:
print(clipValue)

In [ ]:
# Load an image and select some bands of interest.
image = ee.Image('projects/mapbiomas-workspace/public/collection6/mapbiomas_collection60_integration_v1')\
          .select(['classification_2019'])

# Reduce the image to get a one-band maximum value image.
maxValue = image.reduce(ee.Reducer.max())

# Display the result
vis_param = {'max': 13000, 'gamma': 1.6}
center =  [-2.145746, -59.004845]

maxValue_tk = maxValue.getMapId()
Mapdisplay(center,{'Landsat 5':maxValue_tk},zoom_start=4)

# 4. Estatísticas de uma região de imagem

Suponha que haja necessidade de calcular estatísticas sobre uma região (ou regiões) de um `ee.Image`. Para obter estatísticas de valores de pixel em uma região de imagem, use `image.reduceRegion()`. Isso reduz todos os pixels na(s) região(ões) a uma estatística ou outra representação compacta dos dados de pixel na região (por exemplo, histograma). A região é representada como uma `Geometria`, que pode ser um polígono, contendo muitos pixels, ou pode ser um único ponto, caso em que haverá apenas um pixel na região. Em ambos os casos, conforme ilustrado na Figura 2, a saída é uma estatística derivada dos pixels na região.

<center>
<image src=" https://developers.google.com/earth-engine/images/Reduce_region_diagram.png" >
</center>
  

<center>
  Figure 2. An illustration of an `ee.Reducer` applied to an image and a region.
</center>
  
  
  

In [ ]:
# Load input imagery: Landsat 7 5-year composite.
image = ee.Image('projects/mapbiomas-workspace/public/collection6/mapbiomas_collection60_integration_v1')\
        .select(['classification_2019'])

# Load an input region: Sierra Nevada mixed conifer forest.
region = ee.Feature(ee.FeatureCollection(
  'projects/ee-alinephysics/assets/amaz190823')
  .first())
gm = ee.FeatureCollection(
  'projects/ee-alinephysics/assets/amaz190823')

# Reduce the region. The region parameter is the Feature geometry.
meanDictionary = image.reduceRegion(**{
  'reducer': ee.Reducer.median(),
  'geometry': gm,
  'scale': 30,
  'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
pprint(meanDictionary.getInfo())

Observe que neste exemplo a redução é especificada fornecendo o `reducer` (`ee.Reducer.mean()`), a `geometry` (`region.geometry()`), a `scale` (30 metros) e `maxPixels` para o número máximo de pixels a serem inseridos no redutor. Uma escala deve sempre ser especificada nas chamadas de reduceRegion(). Isso ocorre porque em fluxos de processamento complexos, que podem envolver dados de diferentes fontes com diferentes escalas, a escala da saída não será determinada de forma inequívoca a partir das entradas. Nesse c…e, que geralmente produz resultados insatisfatórios. Consulte [esta página](https://developers.google.com/earth-engine/scale) para obter mais informações sobre como o Earth Engine lida com a escala. Há duas maneiras de definir a escala: especificando o parâmetro de escala ou especificando uma transformação CRS e CRS. (Consulte o [glossário](https://developers.google.com/earth-engine/glossary) para obter mais informações sobre CRSs e transformações de CRS). Por exemplo, a redução meanDictionary (acima) é equivalente ao seguinte:

In [ ]:
# As an alternative to specifying scale, specify a CRS and a CRS transform.
# Make this array by constructing a 4326 projection at 30 meters,
# then copying the bounds of the composite, from composite.projection().
affine = [0.00026949458523585647, 0, -180, 0, -0.00026949458523585647, 86.0000269494563];

# Perform the reduction, print the result.
pprint(image.reduceRegion(**{
  'reducer': ee.Reducer.mean(),
  'geometry': gm,
  'crs': 'EPSG:4326',
  'crsTransform': affine,
  'maxPixels': 1e9
}).getInfo())

Em geral, especificar a escala é suficiente e resulta em um código mais legível. O Earth Engine determina quais pixels devem ser inseridos no redutor primeiro rasterizando a região. Se uma escala for especificada sem um CRS, a região será rasterizada na projeção nativa da imagem dimensionada para a resolução especificada. Se um CRS e uma escala forem especificados, a região será rasterizada com base neles. Os pixels estão “na” região se seu centroide estiver coberto pela região na escala e projeção especificadas.

O parâmetro `maxPixels` é necessário para que a computação seja bem-sucedida. Se este parâmetro for deixado de fora do exemplo, um erro será retornado, algo como:



```
Dictionary (Error)
  Image.reduceRegion: Too many pixels in the region. Found 527001545, but only 10000000 allowed.
```


Existem várias opções para superar esses erros: aumentar maxPixels, como no exemplo, aumentar a escala ou definir bestEffort como true, que calcula automaticamente uma nova escala (maior) de modo que maxPixels não seja excedido. Se você não especificar maxPixels, o valor padrão será usado.

# 5. Estatísticas de vizinhanças de imagens

Em vez de especificar uma região sobre a qual realizar uma redução, também é possível especificar uma vizinhança na qual aplicar um redutor. Para reduzir as vizinhanças da imagem, use `image.reduceNeighborhood()`. Neste caso, a redução ocorrerá em uma janela deslizante sobre a imagem de entrada, com o tamanho e formato da janela especificados por um `ee.Kernel`. A saída de `reduceNeighborhood()` será outra imagem, com cada valor de pixel representando a saída da redução em uma vizinhança ao redor daquele pixel na imagem de entrada. A Figura 3 ilustra esse tipo de redução.

<center>
<img src="https://developers.google.com/earth-engine/images/Reduce_Neighborhood.png">  
</center>  

<center>
Figure 3. Illustration of `reduceNeighborhood()`, where the reducer is applied in a kernel.
</center>  

Por exemplo, considere o uso de imagens do National Agriculture Imagery Program (NAIP) para quantificar as diferenças da paisagem resultantes da extração de madeira nas florestas de sequoias da Califórnia. Especificamente, use o desvio padrão (SD) em uma vizinhança para representar a diferença de textura entre a área explorada e a área protegida. Por exemplo, para obter a textura de uma imagem NAIP Normalized Difference Vegetation Index (NDVI), use reduceNeighborhood() para calcular SD em uma vizinhança definida por um kernel:



In [ ]:
# # Define a region in the redwood forest.
# redwoods = ee.Geometry.Rectangle(-124.0665, 41.0739, -123.934, 41.2029)

# # Load input NAIP imagery and build a mosaic.
# naipCollection = ee.ImageCollection('USDA/NAIP/DOQQ')\
#                    .filterBounds(redwoods)\
#                    .filterDate('2012-01-01', '2012-12-31')

# naip = naipCollection.mosaic()

# # Compute NDVI from the NAIP imagery.
# naipNDVI = naip.normalizedDifference(['N', 'R'])

# Compute standard deviation (SD) as texture of the NDVI.
texture = image.reduceNeighborhood(**{
  'reducer': ee.Reducer.stdDev(),
  'kernel': ee.Kernel.circle(7),
})

# Display the results.

# dicc ={'NAIP input imagery': naip.getMapId(),
#        'NDVI': naipNDVI.getMapId({'min': -1, 'max': 1, 'palette': ['FF0000', '00FF00']}),
#        'SD of NDVI':texture.getMapId({'min': 0, 'max': 0.3})}

dicc = { 'SD of NDVI':texture.getMapId()}
Mapdisplay(center,dicc,zoom_start=4)

In [ ]:
print(texture.getMapId())

# 9. Reduções Agrupadas e Estatísticas Zonais

Você pode obter estatísticas em cada zona de uma `Image` ou `FeatureCollection` usando `reducer.group()` para agrupar a saída de um redutor pelo valor de uma entrada especificada. Por exemplo, para calcular a população total e o número de unidades habitacionais em cada estado, este exemplo agrupa a saída de uma redução de um `FeatureCollection` dos condados da seguinte forma:




In [ ]:
# Load a collection of US counties with census data properties.
counties = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM')

# Compute sums of the specified properties, grouped by state name.
sums = counties.filter(ee.Filter.And(ee.Filter.neq('Census 2000 Population', None),
                                     ee.Filter.neq('Census 2000 Housing Units', None)))\
               .reduceColumns(**{
                  'selectors': ['Census 2000 Population', 'Census 2000 Housing Units', 'StateName'],
                  'reducer': ee.Reducer.sum().repeat(2).group(**{
                      'groupField': 2,
                      'groupName': 'state'})})

# Print the resultant Dictionary.
pprint(sums.getInfo())

O argumento `groupField` é o índice da entrada no array seletores que contém os códigos pelos quais agrupar, o argumento `groupName` especifica o nome da propriedade para armazenar o valor da variável de agrupamento. Como o redutor não é repetido automaticamente para cada entrada, a chamada `repeat(2)` é necessária.

Para agrupar a saída de `image.reduceRegions()` você pode especificar uma banda de agrupamento que define grupos por valores de pixel inteiros. Esse tipo de cálculo às vezes é chamado de "estatística zonal", onde as zonas são especificadas como a banda de agrupamento e a estatística é determinada pelo redutor. No exemplo a seguir, a mudança nas luzes noturnas nos Estados Unidos é agrupada por categoria de cobertura do solo:

In [ ]:
# # Load a region representing the United States
# region = ee.Feature(ee.FeatureCollection('ft:1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw')\
#                       .filter(ee.Filter.eq('Country', 'United States'))\
#                       .first())

# # Load MODIS land cover categories in 2001.
# landcover = ee.Image('MODIS/051/MCD12Q1/2001_01_01')\
#               .select('Land_Cover_Type_1') # Select the IGBP classification band.

# # Load nightlights image inputs.
# nl2001 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F152001')\
#            .select('stable_lights')

# nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')\
#            .select('stable_lights')

# # Compute the nightlights decadal difference, add land cover codes.
# nlDiff = nl2012.subtract(nl2001).addBands(landcover)

# Grouped a mean reducer: change of nightlights by land cover category.
means = image.reduceRegion(**{
  'reducer': ee.Reducer.mean(),
  'geometry': gm,
  'scale': 30,
  'maxPixels': 1e8
});

# Print the resultant Dictionary.
pprint(means.getInfo())

Observe que neste exemplo, o `groupField` é o índice da banda que contém as zonas pelas quais agrupar a saída. A primeira banda é o índice 0, a segunda é o índice 1, etc.

# 10. Reduções ponderadas

Por padrão, os redutores aplicados às imagens ponderam as entradas de acordo com o valor da máscara. Isso é relevante no contexto de pixels fracionários criados por meio de operações como `clip()`. Ajuste esse comportamento chamando `unweighted()` no redutor. O uso de um redutor não ponderado força todos os pixels na região a terem o mesmo peso. O exemplo a seguir ilustra como a ponderação do pixel pode afetar a saída do redutor:



In [ ]:
# Load a Landsat 8 input image.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Creat an arbitrary region.
geometry = ee.Geometry.Rectangle(-122.496, 37.532, -121.554, 37.538)

# Make an NDWI image.  It will have one band named 'nd'.
ndwi = image.normalizedDifference(['B3', 'B5']);

# Compute the weighted mean of the NDWI image clipped to the region.
weighted = ndwi.clip(geometry)\
               .reduceRegion(**{'reducer': ee.Reducer.sum(),
                              'geometry': geometry,
                              'scale': 30})\
               .get('nd')

# Compute the UN-weighted mean of the NDWI image clipped to the region.
unweighted = ndwi.clip(geometry)\
                 .reduceRegion(**{'reducer': ee.Reducer.sum().unweighted(),
                                'geometry': geometry,
                                'scale': 30}).get('nd')

# Observe the difference between weighted and unweighted reductions.
print('weighted:', weighted.getInfo())
print('unweighted', unweighted.getInfo())


A diferença nos resultados é devido aos pixels na borda da região que recebem um peso de um como resultado de chamar `unweighted()` no redutor. Para obter uma saída explicitamente ponderada, é preferível definir os pesos explicitamente com `splitWeights()` chamado no redutor.

Um redutor modificado por `splitWeights()` recebe duas entradas, onde a segunda entrada é o peso. O exemplo a seguir ilustra `splitWeights()` calculando a média ponderada do Índice de Vegetação por Diferença Normalizada (NDVI) em uma região, com os pesos fornecidos pela pontuação da nuvem (quanto mais nublado, menor o peso):

In [ ]:

# Use a mean reducer.
reducer = ee.Reducer.median()

# Compute the unweighted mean.
unweighted = image.reduceRegion(reducer, gm, 30)

# compute mean weighted by cloudiness.
weighted = image.reduceRegion(reducer, gm, 30);

# Observe the difference as a result of weighting by cloudiness.
print('unweighted:', unweighted.getInfo())
print('weighted:', weighted.getInfo())

Observe que cloudWeight precisa ser adicionado como uma banda antes de chamar reduceRegion(). O resultado indica que o NDVI médio estimado é maior em função da diminuição do peso dos pixels nublados.




In [ ]:
imagem= ee.Image('projects/ee-alinephysics/assets/mapbiomas-brazil-collection-60-amazonialegal-2019')
center =  [-2.145746, -59.004845]
vizParams = {
    'bands': 'classification_2019'
        }

dicc =  {'mapbiomas mosaic': imagem.getMapId()}

Mapdisplay(center,dicc,zoom_start=4)



# Export Shapefile para Asset GEE

 Conversão Shapefile e CSV to ee.FeaturedCollection

In [ ]:
# From https://bikeshbade.com.np/tutorials/Detail/?title=Geo-pandas%20data%20frame%20to%20GEE%20feature%20collection%20using%20Python&code=13

#Feature to EE
def feature2ee(file):

    #Exception handler
    try:

        #check if the file is shapefile or CSV
        if file.endswith('.shp'):
            gdf = gpd.read_file(file, encoding="utf-8")
            g = [i for i in gdf.geometry]
            features=[]


            #for Polygon geo data type
            if (gdf.geom_type[0] == 'Polygon'):
                for i in range(len(g)):
                    g = [i for i in gdf.geometry]
                    x,y = g[i].exterior.coords.xy
                    cords = np.dstack((x,y)).tolist()

                    g=ee.Geometry.Polygon(cords)
                    feature = ee.Feature(g)
                    features.append(feature)
                print("done")

                ee_object = ee.FeatureCollection(features)

                return ee_object


            #for LineString geo data type
            elif (gdf.geom_type[0] == 'LineString'):
                for i in range(len(g)):
                    g = [i for i in gdf.geometry]
                    x,y = g[i].exterior.coords.xy
                    cords = np.dstack((x,y)).tolist()
                    double_list = reduce(lambda x,y: x+y, cords)

                    g=ee.Geometry.LineString(double_list)
                    feature = ee.Feature(g)
                    features.append(feature)
                print("done")

                ee_object = ee.FeatureCollection(features)

                return ee_object


            #for Point geo data type
            elif (gdf.geom_type[0] == 'Point'):
                for i in range(len(g)):
                    g = [i for i in gdf.geometry]
                    x,y = g[i].exterior.coords.xy
                    cords = np.dstack((x,y)).tolist()
                    double_list = reduce(lambda x,y: x+y, cords)
                    single_list = reduce(lambda x,y: x+y, double_list)

                    g=ee.Geometry.Point(single_list)
                    feature = ee.Feature(g)
                    features.append(feature)
                print("done")

                ee_object = ee.FeatureCollection(features)

                return ee_object


        #check if the file is shapefile or CSV
        #for CSV we need to have file with longitude and latitude
        elif file.endswith('.csv'):
            df = pd.read_csv(file)
            features=[]
            for i in range(df.shape[0]):
                x,y = df.longitude[i],df.latitude[i]
                latlong =[x,y]
                g=ee.Geometry.Point(latlong)
                feature = ee.Feature(g)
                features.append(feature)
            print("done")

            ee_object = ee.FeatureCollection(features)
            return ee_object


    except:

        print("Wrong file format")

Export Feature Collection

In [ ]:
# Export an ee.FeatureCollection as an Earth Engine asset.
def do_upload_shapefile_to_asset(file_path_list):

  for file_path in file_path_list:
    # file_name = file_path[]
    feature = feature2ee(file_path)

    assetId = 'projects/ee-alinephysics/assets/SemanaVeraoOCO-2/' + filename

    task = ee.batch.Export.table.toAsset(**{
        'collection': feature,
        'description': 'exportOcoDataToAssetPath',
        'assetId': 'projects/ee-alinephysics/assets/SemanaVeraoOCO-2/'
    })
    task.start()

    while task.active():
      print('Polling for task (id: {}).'.format(task.id))
      time.sleep(5)

# Alterações na pasta de Assets do Google Earth Engin e Extração de dados do Mapbiomas para cada ponto do dados de CO2 do OCO-2

In [ ]:
center =  [-2.145746, -59.004845]
vizParams = {
    'bands': 'classification_2019'
        }

dicc =  {'mapbiomas mosaic': mapbiomas.getMapId(vizParams)}

Mapdisplay(center,dicc,zoom_start=4)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TESTE_MAPBIOMASOCO/MapBiomas-oco2-teste.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df = df.drop(columns=['system:index', '.geo'])

    -- Pegar os arquivos CSV de um dia chuvoso
    -- Transformar o arquivo CSV em um arquivo SHP
    -- Salvar em assets no GEE

    -- Fazer uma função que com a lista de paths no asset, acessa cada path, realiza a junção com os dados do mapbiomas da classificacao de 2019 e salva o arquivo CSV de cada data.

    --Depois cria outra função que realizar a junção dos csvs de cada dia.

In [ ]:
path_files_oco2_inv = get_list_csv_files('/content/drive/MyDrive/Dados_OCO-2/csv/Inverno2019')

In [ ]:
path_files_oco2_inv[0][51:57]

##Listagem de Assets

In [ ]:
import ee
ee.Initialize()

def get_asset_list(parent):
    parent_asset = ee.data.getAsset(parent)
    parent_id = parent_asset['name']
    parent_type = parent_asset['type']
    asset_list = []
    child_assets = ee.data.listAssets({'parent': parent_id})['assets']
    for child_asset in child_assets:
        child_id = child_asset['name']
        child_type = child_asset['type']
        if child_type in ['FOLDER','IMAGE_COLLECTION']:
            # Recursively call the function to get child assets
            asset_list.extend(get_asset_list(child_id))
        else:
            asset_list.append(child_id)
    return asset_list

all_assets = get_asset_list('projects/ee-alinephysics/assets/')

print('Found {} assets'.format(len(all_assets)))

In [ ]:
# assets_for_delete_2 = [
#  'projects/ee-alinephysics/assets/oco2_L2StdND_38524a_210928_B10206r_211113182109', 'projects/ee-alinephysics/assets/teste/teste1']

In [ ]:
all_assets

### Extração de Classes de Mapbiomas

In [ ]:
import time

def extract_mapbiomas_class_for_regions(asset, year, shapefile):

  # asset ='projects/mapbiomas-workspace/public/collection6/mapbiomas_collection60_integration_v1'

  # Define the year of your interest
  # year = 2019;

  gm = ee.FeatureCollection(shapefile)
  name = shapefile.rsplit('/', -1)
  name = name[-1]

  mapbiomas = ee.Image(asset).clip(gm)

  reducer = ee.Reducer.median()

  # maineMeansFeatures = image.reduceRegions(**{
  #   'collection': gm,
  #   'reducer': ee.Reducer.mean(),
  #   'scale': 30,
  # })

  reduced = mapbiomas.reduceRegions(**{
    'collection': gm,
    'reducer': reducer,
    'scale': 100000,
    'tileScale': 1
  });

  print(reduced)
  print(name)
  print(type(name))
  filename = 'oco2-mapbiomas-inverno-' + name

  task = ee.batch.Export.table.toDrive(**{
      'collection': reduced,
      'description': 'DadosOCO-2_Mapbiomas-inverno' + filename,
      'folder': 'oco-2_mapbiomas/inverno-2015a2021',
      'fileNamePrefix': filename,
      'fileFormat': 'CSV'
  })
  task.start()

  while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(5)


  # task = ee.batch.Export.imageluscogna/.toDrive(**{
  #     'collection': reduced,
  #     'description': 'DadosOCO-2_Mapbiomas-Chuva',
  #     'folder': 'TESTE_MAPBIOMASOCO',
  #     'fileNamePrefix': 'MapBiomas-oco2-teste',
  #     'fileFormat': 'CSV'
  # })
  # task.start()

  # while task.active():
  #   print('Polling for task (id: {}).'.format(task.id))
  #   time.sleep(5)


In [ ]:
# for shapefile in all_assets:
shapefile = 'projects/ee-alinephysics/assets/Grid_Amazonia_Legal_100km'
asset ='projects/mapbiomas-workspace/public/collection6/mapbiomas_collection60_integration_v1'
year = 2019
extract_mapbiomas_class_for_regions(asset, year, shapefile)


## Mudando assets de local



In [ ]:
import argparse
import ee

parser = argparse.ArgumentParser()
parser.add_argument('--old_collection', help='old collection')
parser.add_argument('--new_collection', help='new collection')
parser.add_argument('--delete', help='delete old collection',
    action=argparse.BooleanOptionalAction)

# args = parser.parse_args()

old_collection = old_collection
new_collection = new_collection

ee.Initialize()

# Check if new collection exists
try:
    ee.ImageCollection(new_collection).getInfo()
except:
    print('Collection {} does not exist'.format(new_collection))
    ee.data.createAsset({'type': ee.data.ASSET_TYPE_IMAGE_COLL}, new_collection)
    print('Created a new empty collection {}.'.format(new_collection))


assets = ee.data.listAssets({'parent': old_collection})['assets']


for asset in assets:
    old_name = asset['name']
    new_name = old_name.replace(old_collection, new_collection)
    print('Copying {} to {}'.format(old_name, new_name))
    ee.data.copyAsset(old_name, new_name, True)
    if args.delete:
        print('Deleting <{}>'.format(old_name))
        ee.data.deleteAsset(old_name)

if args.delete:
    print('Deleting Collection <{}>'.format(old_collection))
    ee.data.deleteAsset(old_collection)

##Exclusão de Assets

In [ ]:
ee.data.deleteAsset('projects/ee-alinephysics/assets/bdgeodata/20082019')

In [ ]:
for asset in assets_for_delete_2:
  ee.data.deleteAsset(asset)

In [ ]:
# "projects/ee-alinephysics/assets/bdgeodata/"

##Criação de shapefile e Salva em Assets

In [ ]:
import ee
import os
import geemap
import pandas as pd
import geopandas as gpd


def create_geodataframe_and_save_in_assets(df, path_name, asset_name):
  # create a tmp gdf
  gdf = gpd.GeoDataFrame(
      df,
      crs='EPSG:4326',
      geometry = gpd.points_from_xy(
          df['longitude'],
          df['latitude']
      )
  )

  # convert it into geo-json
  json_df = json.loads(gdf.to_json())

  # create a gee object with geemap
  ee_object = geemap.geojson_to_ee(json_df)

  # upload this object to earthengine
  asset = os.path.join(path_name, asset_name)

  #create and launch the task
  task_config = {
      'collection': ee_object,
      'description':asset_name,
      'assetId': asset
  }
  task = ee.batch.Export.table.toAsset(**task_config)
  task.start()
  return "Success"

In [ ]:
def save_object_in_asset(path):
  for path_file in path_files_oco2_inv:
    if '0106a3109' not in path_file:
      asset_name = path_file[51:57]
      print(asset_name)
      dataframe = pd.read_csv(path_file)
      create_geodataframe_and_save_in_assets(dataframe, path, asset_name)
  return "Ok"

In [ ]:
path_inv = 'projects/ee-alinephysics/assets/bdgeodata/'
save_object_in_asset(path_inv)

In [ ]:
lista_bioma_shp = ['projects/ee-alinephysics/assets/grid_1_Bioma_Amazon_0-25deg',
 'projects/ee-alinephysics/assets/grid_2_Bioma_Amazon_0-25deg',
 'projects/ee-alinephysics/assets/grid_3_Bioma_Amazon_0-25deg',
 'projects/ee-alinephysics/assets/grid_4_Bioma_Amazon_0-25deg']

In [ ]:
import ee
import geemap

# Inicializar a API GEE
ee.Initialize()

# Carregar o shapefile
shapefile = ee.FeatureCollection('projects/ee-alinephysics/assets/grid_1_Bioma_Amazon_0-25deg')

# Definir a região de interesse com base no shapefile
region = shapefile.geometry().bounds()

# Definir uma resolução espacial de 0,25°
scale = 926.62543305555556  # 0,25° em metros (1° = 111320m)

# Função para adicionar os dados da imagem à feição do shapefile
def addDataToFeature(feature):
    feature = ee.Feature(feature)
    feature = feature.set(image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=scale
    ))
    return feature

# Carregar a coleção de imagens MODIS
collection = ee.ImageCollection('MODIS/006/MOD13A1').filterBounds(region)

# Filtrar as imagens dentro de um período específico (opcional)
start_date = ee.Date('2022-01-01')
end_date = ee.Date('2022-12-31')
collection = collection.filterDate(start_date, end_date)

# Reduzir as imagens para uma única imagem usando a média
image = collection.mean()

# Mapear a função addDataToFeature sobre cada feição do shapefile
output = shapefile.map(addDataToFeature)

# Exportar o resultado para um arquivo CSV
geemap.extract_values_to_points(output, 'output.csv')


# Extração dos dados de Precipitação Imerg
ee.ImageCollection("NASA/GPM_L3/IMERG_V06")

In [ ]:

# Define the year of your interest
year = 2019;
gm = ee.FeatureCollection(
  'projects/ee-alinephysics/assets/amaz190823')

mapbiomas = ee.ImageCollection("NASA/GPM_L3/IMERG_V06").clip(gm)

reducer = ee.Reducer.median()

# maineMeansFeatures = image.reduceRegions(**{
#   'collection': gm,
#   'reducer': ee.Reducer.mean(),
#   'scale': 30,
# })

reduced = mapbiomas.reduceRegions(**{
  'collection': gm,
  'reducer': reducer,
  'scale': 30,
  'tileScale': 1
});

print(reduced)

# task = ee.batch.Export.table.toDrive(**{
#     'collection': reduced,
#     'description': 'DadosOCO-2_Mapbiomas-Chuva',
#     'folder': 'TESTE_MAPBIOMASOCO',
#     'fileNamePrefix': 'MapBiomas-oco2-teste',
#     'fileFormat': 'CSV'
# })
# task.start()

# while task.active():
#   print('Polling for task (id: {}).'.format(task.id))
#   time.sleep(5)


# task = ee.batch.Export.imageluscogna/.toDrive(**{
#     'collection': reduced,
#     'description': 'DadosOCO-2_Mapbiomas-Chuva',
#     'folder': 'TESTE_MAPBIOMASOCO',
#     'fileNamePrefix': 'MapBiomas-oco2-teste',
#     'fileFormat': 'CSV'
# })
# task.start()

# while task.active():
#   print('Polling for task (id: {}).'.format(task.id))
#   time.sleep(5)


In [ ]:
oco2_map = pd.read_csv('/content/oco2-mapbiomas-inverno-Grid_Amazonia_Legal_100km.csv')

In [ ]:
oco2_map.head(5)

#Download dos dados do GEE para assets e drive

In [ ]:
def get_image_for_band_and_clip(image, geometry):
  image_clipped =  image.clip(geometry).copy_properties(image, ["system:time_start"])
  return image_clipped

def get_terra_climate_collection(path, start_date, end_date, geometry):
  mosaic = ee.ImageCollection(path).filter(ee.Filter.date(start_date, end_date)).filter(ee.Filter.bounds(geometry)).map(get_image_for_band_and_clip).mosaic()
  return mosaic

def get_terra_climate_image(path, geometry):
  image = ee.Image(path).clip(geometry)
  return image

def create_list_of_years():
  years = []
  initial_year = 2020
  final_year = 2021
  for year in range(initial_year, final_year):
    years.append(str(year))
  return years

def create_list_of_months():
  months = []
  for month in range(1,13):
    month_st = str(month)
    if month < 10:
      month_st = '0' + month_st
    months.append(month_st)
  return months

def get_terra_climate_image_month_years(years, months):
  base_path = 'IDAHO_EPSCOR/TERRACLIMATE/'
  ym_list = []
  for year in years:
    for month in months:
      ym_list.append(base_path + year + month)
  return ym_list

def export_image_to_drive(mosaic, description, folder_name, scale, geometry):
  task_config = {
      'image': mosaic,
      'description': description,
      'folder': folder_name,
      'scale': scale,
      'region': geometry.geometry(),
      'crs': 'EPSG:4674',
      'fileFormat': 'GeoTIFF'
  }
  task = ee.batch.Export.image.toDrive(**task_config)
  task.start()

def export_image_to_asset(image, file_name, asset, scale, geometry):
  task_config = {
      'assetId': asset,
      'image': image,
      'description': file_name,
      'scale': scale,
      'region': geometry.geometry(),
      'crs': 'EPSG:4674'
  }
  task = ee.batch.Export.image.toAsset(**task_config)
  task.start()


def create_filename(initial_path):
  year_path = initial_path[26:]
  folder_path = 'projects/ee-alinephysics/assets/terraclimate-mt/image/'
  final_path = folder_path + year_path
  return final_path, year_path

def get_month_year_and_download_image_to_asset(years, months, geometry):
  path_list = get_terra_climate_image_month_years(years, months)

  for path in path_list:
    image = get_terra_climate_image(path, geometry)
    asset, file_name = create_filename(path)
    scale = 4638.3
    export_image_to_asset(image, file_name, asset, scale, geometry)

def get_month_year_and_download_image_to_drive(years, months, geometry):
  path_list = get_terra_climate_image_month_years(years, months)

  for path in path_list:
    image = get_terra_climate_image(path, geometry)
    asset, file_name = create_filename(path)
    folder_name = 'terraclimate-mt'
    scale = 4638.3
    export_image_to_drive(image, file_name, folder_name, scale, geometry)

In [ ]:
years = create_list_of_years()
months = create_list_of_months()

geometry = ee.FeatureCollection('projects/ee-alinephysics/assets/terraclimate-mt/MT_UF_2021')

get_month_year_and_download_image_to_asset(years, months, geometry)

In [ ]:
years = create_list_of_years()
months = create_list_of_months()

geometry = ee.FeatureCollection('projects/ee-alinephysics/assets/terraclimate-mt/MT_UF_2021')

get_month_year_and_download_image_to_drive(years, months, geometry)

### MODIS

In [ ]:
import time

def extract_modis_data_lai_for_grid(asset, shapefile):

  # asset ='projects/mapbiomas-workspace/public/collection6/mapbiomas_collection60_integration_v1'

  # Define the year of your interest
  # year = 2019;

  gm = ee.FeatureCollection(shapefile)

  modis_data = ee.Image(asset).clip(gm)

  reducer = ee.Reducer.mean()

  # maineMeansFeatures = image.reduceRegions(**{
  #   'collection': gm,
  #   'reducer': ee.Reducer.mean(),
  #   'scale': 30,
  # })

  reduced = modis_data.reduceRegions(**{
    'collection': gm,
    'reducer': reducer,
    'scale': 100000,
    'tileScale': 1
  });

  print(reduced)

  filename = 'modis-100km-lai-fpar' + asset[19:30]

  task = ee.batch.Export.table.toDrive(**{
      'collection': reduced,
      'description': filename,
      'folder': 'MODIS',
      'fileNamePrefix': filename,
      'fileFormat': 'CSV'
  })
  task.start()

  while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(5)


  # task = ee.batch.Export.imageluscogna/.toDrive(**{
  #     'collection': reduced,
  #     'description': 'DadosOCO-2_Mapbiomas-Chuva',
  #     'folder': 'TESTE_MAPBIOMASOCO',
  #     'fileNamePrefix': 'MapBiomas-oco2-teste',
  #     'fileFormat': 'CSV'
  # })
  # task.start()

  # while task.active():
  #   print('Polling for task (id: {}).'.format(task.id))
  #   time.sleep(5)

In [ ]:
lai_data = ['MODIS/061/MOD15A2H/2015_08_05'
,'MODIS/061/MOD15A2H/2015_08_13','MODIS/061/MOD15A2H/2015_08_21','MODIS/061/MOD15A2H/2015_08_29','MODIS/061/MOD15A2H/2015_09_06','MODIS/061/MOD15A2H/2015_09_14','MODIS/061/MOD15A2H/2015_09_22','MODIS/061/MOD15A2H/2015_09_30','MODIS/061/MOD15A2H/2016_08_04','MODIS/061/MOD15A2H/2016_08_12','MODIS/061/MOD15A2H/2016_08_20','MODIS/061/MOD15A2H/2016_08_28','MODIS/061/MOD15A2H/2016_09_05','MODIS/061/MOD15A2H/2016_09_13','MODIS/061/MOD15A2H/2016_09_21','MODIS/061/MOD15A2H/2016_09_29','MODIS/061/MOD15A2H/2017_07_28','MODIS/061/MOD15A2H/2017_08_05','MODIS/061/MOD15A2H/2017_08_13','MODIS/061/MOD15A2H/2017_08_21','MODIS/061/MOD15A2H/2017_08_29','MODIS/061/MOD15A2H/2017_09_06','MODIS/061/MOD15A2H/2017_09_14','MODIS/061/MOD15A2H/2017_09_22','MODIS/061/MOD15A2H/2017_09_30','MODIS/061/MOD15A2H/2018_07_28','MODIS/061/MOD15A2H/2018_08_05','MODIS/061/MOD15A2H/2018_08_13','MODIS/061/MOD15A2H/2018_08_21','MODIS/061/MOD15A2H/2018_08_29','MODIS/061/MOD15A2H/2018_09_06','MODIS/061/MOD15A2H/2018_09_14','MODIS/061/MOD15A2H/2018_09_22','MODIS/061/MOD15A2H/2018_09_30','MODIS/061/MOD15A2H/2019_07_28','MODIS/061/MOD15A2H/2019_08_05','MODIS/061/MOD15A2H/2019_08_13','MODIS/061/MOD15A2H/2019_08_21','MODIS/061/MOD15A2H/2019_08_29','MODIS/061/MOD15A2H/2019_09_06','MODIS/061/MOD15A2H/2019_09_14','MODIS/061/MOD15A2H/2019_09_22','MODIS/061/MOD15A2H/2019_09_30','MODIS/061/MOD15A2H/2020_07_27','MODIS/061/MOD15A2H/2020_08_04','MODIS/061/MOD15A2H/2020_08_12','MODIS/061/MOD15A2H/2020_08_20','MODIS/061/MOD15A2H/2020_08_28','MODIS/061/MOD15A2H/2020_09_05','MODIS/061/MOD15A2H/2020_09_13','MODIS/061/MOD15A2H/2020_09_21','MODIS/061/MOD15A2H/2020_09_29','MODIS/061/MOD15A2H/2021_07_28','MODIS/061/MOD15A2H/2021_08_05','MODIS/061/MOD15A2H/2021_08_13','MODIS/061/MOD15A2H/2021_08_21','MODIS/061/MOD15A2H/2021_08_29','MODIS/061/MOD15A2H/2021_09_06','MODIS/061/MOD15A2H/2021_09_14','MODIS/061/MOD15A2H/2021_09_22','MODIS/061/MOD15A2H/2021_09_30']

In [ ]:
shapefile = 'projects/ee-alinephysics/assets/Grid_Amazonia_Legal_100km'

for asset in lai_data:
  extract_modis_data_lai_for_grid(asset, shapefile)

In [ ]:
lai_data[0][19:30]

In [ ]:
veg_data[0][18:30]

In [ ]:
import time

def extract_modis_data_for_grid(asset, shapefile):

  # asset ='projects/mapbiomas-workspace/public/collection6/mapbiomas_collection60_integration_v1'

  # Define the year of your interest
  # year = 2019;

  gm = ee.FeatureCollection(shapefile)

  modis_data = ee.Image(asset).clip(gm)

  reducer = ee.Reducer.mean()

  # maineMeansFeatures = image.reduceRegions(**{
  #   'collection': gm,
  #   'reducer': ee.Reducer.mean(),
  #   'scale': 30,
  # })

  reduced = modis_data.reduceRegions(**{
    'collection': gm,
    'reducer': reducer,
    'scale': 100000,
    'tileScale': 1
  });

  print(reduced)

  filename = 'modis-100km-evi-ndvi' + asset[18:30]

  task = ee.batch.Export.table.toDrive(**{
      'collection': reduced,
      'description': filename,
      'folder': 'MODIS/',
      'fileNamePrefix': filename,
      'fileFormat': 'CSV'
  })
  task.start()

  while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(5)


  # task = ee.batch.Export.imageluscogna/.toDrive(**{
  #     'collection': reduced,
  #     'description': 'DadosOCO-2_Mapbiomas-Chuva',
  #     'folder': 'TESTE_MAPBIOMASOCO',
  #     'fileNamePrefix': 'MapBiomas-oco2-teste',
  #     'fileFormat': 'CSV'
  # })
  # task.start()

  # while task.active():
  #   print('Polling for task (id: {}).'.format(task.id))
  #   time.sleep(5)

In [ ]:
veg_data = ['MODIS/061/MOD13A1/2015_07_28','MODIS/061/MOD13A1/2015_08_13','MODIS/061/MOD13A1/2015_08_29','MODIS/061/MOD13A1/2015_09_14','MODIS/061/MOD13A1/2015_09_30','MODIS/061/MOD13A1/2015_10_16',

'MODIS/061/MOD13A1/2016_08_12','MODIS/061/MOD13A1/2016_08_28','MODIS/061/MOD13A1/2016_09_13','MODIS/061/MOD13A1/2016_09_29',

'MODIS/061/MOD13A1/2017_07_28','MODIS/061/MOD13A1/2017_08_13','MODIS/061/MOD13A1/2017_08_29','MODIS/061/MOD13A1/2017_09_14','MODIS/061/MOD13A1/2017_09_30',

'MODIS/061/MOD13A1/2018_07_28','MODIS/061/MOD13A1/2018_08_13','MODIS/061/MOD13A1/2018_08_29','MODIS/061/MOD13A1/2018_09_14','MODIS/061/MOD13A1/2018_09_30',

'MODIS/061/MOD13A1/2019_07_28','MODIS/061/MOD13A1/2019_08_13','MODIS/061/MOD13A1/2019_08_29','MODIS/061/MOD13A1/2019_09_14','MODIS/061/MOD13A1/2019_09_30',

'MODIS/061/MOD13A1/2020_07_27','MODIS/061/MOD13A1/2020_08_12','MODIS/061/MOD13A1/2020_08_28','MODIS/061/MOD13A1/2020_09_13','MODIS/061/MOD13A1/2020_09_29',

'MODIS/061/MOD13A1/2021_07_28','MODIS/061/MOD13A1/2021_08_13','MODIS/061/MOD13A1/2021_08_29','MODIS/061/MOD13A1/2021_09_14','MODIS/061/MOD13A1/2021_09_30']

In [ ]:
shapefile = 'projects/ee-alinephysics/assets/Grid_Amazonia_Legal_100km'

for asset in veg_data:
  print(asset)
  extract_modis_data_for_grid(asset, shapefile)

In [ ]:
def create_date_modis_reflectance():
  dates = []
  for year in ['2015','2016','2017','2018','2019','2020','2021']:
    for month in ['08','09']:
      for day in range(1,32):
        if day < 10:
          date = year + '_' + month + '_0' + str(day)
          dates.append(date)
        else:
          date = year + '_' + month + '_' + str(day)
          dates.append(date)
  return dates

In [ ]:
dates = create_date_modis_reflectance()

In [ ]:
dates[0:5]

In [ ]:
import time

def extract_modis_data_reflectance_for_grid(asset, shapefile):


  gm = ee.FeatureCollection(shapefile)

  modis_data = ee.Image(asset).clip(gm)

  reducer = ee.Reducer.mean()

  # maineMeansFeatures = image.reduceRegions(**{
  #   'collection': gm,
  #   'reducer': ee.Reducer.mean(),
  #   'scale': 30,
  # })

  reduced = modis_data.reduceRegions(**{
    'collection': gm,
    'reducer': reducer,
    'scale': 100000,
    'tileScale': 1
  });

  print(reduced)

  filename = 'modis-100km-reflectance' + asset[17:34]

  task = ee.batch.Export.table.toDrive(**{
      'collection': reduced,
      'description': 'modisreflectance',
      'folder': 'MODISReflectance',
      'fileNamePrefix': filename,
      'fileFormat': 'CSV'
  })
  task.start()

  while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(5)


  # task = ee.batch.Export.imageluscogna/.toDrive(**{
  #     'collection': reduced,
  #     'description': 'DadosOCO-2_Mapbiomas-Chuva',
  #     'folder': 'TESTE_MAPBIOMASOCO',
  #     'fileNamePrefix': 'MapBiomas-oco2-teste',
  #     'fileFormat': 'CSV'
  # })
  # task.start()

  # while task.active():
  #   print('Polling for task (id: {}).'.format(task.id))
  #   time.sleep(5)

In [ ]:
asset[17:34]

In [ ]:
shapefile = 'projects/ee-alinephysics/assets/Grid_Amazonia_Legal_100km'

for date in dates:
  asset = 'MODIS/006/MCD43A4/' + date
  extract_modis_data_reflectance_for_grid(asset, shapefile)

# Download MODIS e ERA5 data para célula de Km67

In [ ]:
pip install geopandas

In [ ]:
# Importar bibliotecas
import ee
import geopandas as gpd
import pandas as pd
from google.colab import drive

# Autenticação no Google Earth Engine
ee.Authenticate()
ee.Initialize()

In [ ]:
grade_asset = ee.FeatureCollection('projects/ee-alinephysics/assets/grid_cells_Bioma_Amazon_025deg')
grade_asset

In [ ]:
# Salvar o dataframe de resultados como CSV
df_resultado.to_csv('resultado.csv', index=False)

# Montar o drive do Google para salvar o arquivo CSV
drive.mount('/content/drive')

# Salvar o arquivo CSV no drive
!cp resultado.csv "/content/drive/My Drive/"

In [ ]:
bioma_1 = gpd.read_file('/content/drive/MyDrive/bioma-amazonia/bioma_amazonia_0.25deg/grid_1_Bioma_Amazon_0.25deg.shp')
bioma_1.head(4)

In [ ]:
# Salvar o dataframe de resultados como CSV
df_resultado.to_csv('resultado.csv', index=False)

# Montar o drive do Google para salvar o arquivo CSV
drive.mount('/content/drive')

# Salvar o arquivo CSV no drive
!cp resultado.csv "/content/drive/My Drive/"

In [ ]:
test_modis = pd.read_csv('/content/drive/MyDrive/bioma_amazonia_modis_test/MODIS_061_LAI_FPAR_100km_2015.csv')
test_modis.head()

#Extract MODIS e ERA5 data


In [ ]:
["dewpoint_temperature_2m","temperature_2m","skin_temperature","soil_temperature_level_1","soil_temperature_level_2","soil_temperature_level_3","soil_temperature_level_4","lake_bottom_temperature","lake_ice_depth","lake_ice_temperature",
"lake_mix_layer_depth",
"lake_mix_layer_temperature",
"lake_shape_factor",
"lake_total_layer_temperature",
"volumetric_soil_water_layer_1",
"volumetric_soil_water_layer_2",
"volumetric_soil_water_layer_3",
"volumetric_soil_water_layer_4",
"forecast_albedo",
"surface_latent_heat_flux_sum",
"surface_net_solar_radiation_sum",
"surface_net_thermal_radiation_sum",
"surface_sensible_heat_flux_sum",
"surface_solar_radiation_downwards_sum",
"surface_thermal_radiation_downwards_sum",
"evaporation_from_bare_soil_sum",
"evaporation_from_open_water_surfaces_excluding_oceans_sum",
"evaporation_from_the_top_of_canopy_sum",
"evaporation_from_vegetation_transpiration_sum",
"potential_evaporation_sum",
"runoff_sum",
"snow_evaporation_sum",
"sub_surface_runoff_sum",
"surface_runoff_sum",
"total_evaporation_sum",
"u_component_of_wind_10m",
"v_component_of_wind_10m",
"surface_pressure",
"total_precipitation_sum",
"leaf_area_index_high_vegetation",
"leaf_area_index_low_vegetation"]

In [ ]:
"dewpoint_temperature_2m","temperature_2m","skin_temperature","soil_temperature_level_1","soil_temperature_level_2","soil_temperature_level_3","soil_temperature_level_4", "volumetric_soil_water_layer_1",  "volumetric_soil_water_layer_2",
"volumetric_soil_water_layer_3",  "volumetric_soil_water_layer_4","evaporation_from_bare_soil_sum",  "evaporation_from_open_water_surfaces_excluding_oceans_sum",  "evaporation_from_the_top_of_canopy_sum",  "evaporation_from_vegetation_transpiration_sum",  "potential_evaporation_sum",  "runoff_sum",  "snow_evaporation_sum",  "sub_surface_runoff_sum",  "surface_runoff_sum",  "total_evaporation_sum", "surface_pressure",  "total_precipitation_sum",  "leaf_area_index_high_vegetation",  "leaf_area_index_low_vegetation",  "dewpoint_temperature_2m_min",  "dewpoint_temperature_2m_max",  "temperature_2m_min",  "temperature_2m_max",  "skin_temperature_min",  "skin_temperature_max",  "soil_temperature_level_1_min",
"soil_temperature_level_1_max",  "soil_temperature_level_2_min",  "soil_temperature_level_2_max",  "soil_temperature_level_3_min",  "soil_temperature_level_3_max",  "soil_temperature_level_4_min",  "soil_temperature_level_4_max", "volumetric_soil_water_layer_1_min",  "volumetric_soil_water_layer_1_max",  "volumetric_soil_water_layer_2_min",
"volumetric_soil_water_layer_2_max","volumetric_soil_water_layer_3_min","volumetric_soil_water_layer_3_max","volumetric_soil_water_layer_4_min","volumetric_soil_water_layer_4_max","evaporation_from_bare_soil_min","evaporation_from_bare_soil_max","evaporation_from_open_water_surfaces_excluding_oceans_min","evaporation_from_open_water_surfaces_excluding_oceans_max","evaporation_from_the_top_of_canopy_min","evaporation_from_the_top_of_canopy_max","evaporation_from_vegetation_transpiration_min","evaporation_from_vegetation_transpiration_max","potential_evaporation_min","potential_evaporation_max", "surface_pressure_min",  "surface_pressure_max","total_precipitation_min","total_precipitation_max","leaf_area_index_high_vegetation_min","leaf_area_index_high_vegetation_max","leaf_area_index_low_vegetation_min", "leaf_area_index_low_vegetation_max"]

In [ ]:
"dewpoint_temperature_2m","temperature_2m","skin_temperature","soil_temperature_level_1","soil_temperature_level_2","soil_temperature_level_3","soil_temperature_level_4", "volumetric_soil_water_layer_1",  "volumetric_soil_water_layer_2",
"volumetric_soil_water_layer_3",  "volumetric_soil_water_layer_4","evaporation_from_bare_soil_sum",  "evaporation_from_open_water_surfaces_excluding_oceans_sum",  "evaporation_from_the_top_of_canopy_sum",  "evaporation_from_vegetation_transpiration_sum",  "potential_evaporation_sum",  "runoff_sum",  "snow_evaporation_sum",  "sub_surface_runoff_sum",  "surface_runoff_sum",  "total_evaporation_sum", "surface_pressure",  "total_precipitation_sum",  "leaf_area_index_high_vegetation",  "leaf_area_index_low_vegetation",  "dewpoint_temperature_2m_min",  "dewpoint_temperature_2m_max",  "temperature_2m_min",  "temperature_2m_max",  "skin_temperature_min",  "skin_temperature_max",  "soil_temperature_level_1_min",
"soil_temperature_level_1_max",  "soil_temperature_level_2_min",  "soil_temperature_level_2_max",  "soil_temperature_level_3_min",  "soil_temperature_level_3_max",  "soil_temperature_level_4_min",  "soil_temperature_level_4_max", "volumetric_soil_water_layer_1_min",  "volumetric_soil_water_layer_1_max",  "volumetric_soil_water_layer_2_min",
"volumetric_soil_water_layer_2_max","volumetric_soil_water_layer_3_min","volumetric_soil_water_layer_3_max","volumetric_soil_water_layer_4_min","volumetric_soil_water_layer_4_max","evaporation_from_bare_soil_min","evaporation_from_bare_soil_max","evaporation_from_open_water_surfaces_excluding_oceans_min","evaporation_from_open_water_surfaces_excluding_oceans_max","evaporation_from_the_top_of_canopy_min","evaporation_from_the_top_of_canopy_max","evaporation_from_vegetation_transpiration_min","evaporation_from_vegetation_transpiration_max","potential_evaporation_min","potential_evaporation_max", "surface_pre]

In [ ]:
"dewpoint_temperature_2m","temperature_2m","skin_temperature","soil_temperature_level_1","soil_temperature_level_2","soil_temperature_level_3","soil_temperature_level_4", "volumetric_soil_water_layer_1",  "volumetric_soil_water_layer_2",
"volumetric_soil_water_layer_3",  "volumetric_soil_water_layer_4","evaporation_from_bare_soil_sum",  "evaporation_from_open_water_surfaces_excluding_oceans_sum",  "evaporation_from_the_top_of_canopy_sum",  "evaporation_from_vegetation_transpiration_sum",  "potential_evaporation_sum",  "runoff_sum", "total_evaporation_sum", "surface_pressure",  "total_precipitation_sum",  "leaf_area_index_high_vegetation",  "leaf_area_index_low_vegetation",  "dewpoint_temperature_2m_min",  "dewpoint_temperature_2m_max",  "temperature_2m_min",  "temperature_2m_max", "volumetric_soil_water_layer_1_min",  "volumetric_soil_water_layer_1_max","evaporation_from_the_top_of_canopy_min","evaporation_from_the_top_of_canopy_max","evaporation_from_vegetation_transpiration_min","evaporation_from_vegetation_transpiration_max", "surface_pressure_min","total_precipitation_max","leaf_area_index_high_vegetation_min","leaf_area_index_high_vegetation_max","leaf_area_index_low_vegetation_min", "leaf_area_index_low_vegetation_max"]

In [ ]:
pip install cdsapi

In [ ]:
#!/usr/bin/env python

import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'variable': 'temperature',
        'pressure_level': '1000',
        'year': '2008',
        'month': '01',
        'day': '01',
        'time': '12:00',
        'format': 'netcdf',                 # Supported format: grib and netcdf. Default: grib
    },
    'download.nc')                          # Output file. Adapt as you wish.

In [ ]:
function clipToCol(image) {
  return image.clip(table);
}

var collection = ee.ImageCollection("ECMWF/ERA5/DAILY")
                  .filterDate('2002-01-01', '2012-01-01')
                  .map(clipToCol).mosaic();

print(collection)

var colorizedVis = {
  min: 0,
  max: 100,
  palette: [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
};

Map.setCenter(-58.371867748863146, -6.36, 2);
Map.addLayer(collection.select("dewpoint_temperature_2m"), colorizedVis, 'mean_2m_air_temperature');

var reducer = ee.Reducer.mean()

var reduced = collection.reduceRegions({
  collection: table,
  reducer: reducer,
  scale: 926.62543305555556,
  tileScale: 1
});

// print(reduced)


Export.table.toDrive({
    collection: reduced,
    description: 'bioma_amazonia_era5_gee',
    folder: 'bioma_amazonia_era5_gee',
    fileNamePrefix: 'grid1_bioma_amazonia_era5_gee_2002-2012_0-25degree',
    fileFormat: 'CSV'
})


function clipToCol1(image) {
  return image.clip(table2);
}

var collection1 = ee.ImageCollection("ECMWF/ERA5/DAILY")
                  .filterDate('2002-01-01', '2012-01-01')
                  .map(clipToCol1).mosaic();

print(collection1)

var colorizedVis = {
  min: 0,
  max: 100,
  palette: [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
};


var reducer = ee.Reducer.mean()

var reduced1 = collection1.reduceRegions({
  collection: table2,
  reducer: reducer,
  scale: 926.62543305555556,
  tileScale: 1
});

// print(reduced1)


Export.table.toDrive({
    collection: reduced1,
    description: 'bioma_amazonia_era5_gee',
    folder: 'bioma_amazonia_era5_gee',
    fileNamePrefix: 'grid2_bioma_amazonia_era5_gee_2002-2012_0-25degree',
    fileFormat: 'CSV'
})


function clipToCol2(image) {
  return image.clip(table3);
}

var collection2 = ee.ImageCollection("ECMWF/ERA5/DAILY")
                  .filterDate('2002-07-04', '2012-01-01')
                  .map(clipToCol2).mosaic();

print(collection2)

var colorizedVis = {
  min: 0,
  max: 100,
  palette: [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
};

var reducer = ee.Reducer.mean()

var reduced2 = collection2.reduceRegions({
  collection: table3,
  reducer: reducer,
  scale: 926.62543305555556,
  tileScale: 1
});

// print(reduced2)


Export.table.toDrive({
    collection: reduced2,
    description: 'bioma_amazonia_era5_gee',
    folder: 'bioma_amazonia_era5_gee',
    fileNamePrefix: 'grid3_bioma_amazonia_era5_gee_2002-2012_0-25degree',
    fileFormat: 'CSV'
})



function clipToCol3(image) {
  return image.clip(table4);
}

var collection3 = ee.ImageCollection("ECMWF/ERA5/DAILY")
                  .filterDate('2002-01-01', '2012-01-01')
                  .map(clipToCol3).mosaic();

print(collection3)

var colorizedVis = {
  min: 0,
  max: 100,
  palette: [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
};

var reducer = ee.Reducer.mean()

var reduced3 = collection3.reduceRegions({
  collection: table4,
  reducer: reducer,
  scale: 926.62543305555556,
  tileScale: 1
});

// print(reduced3)


Export.table.toDrive({
    collection: reduced3,
    description: 'bioma_amazonia_era5',
    folder: 'bioma_amazonia_era5_gee',
    fileNamePrefix: 'grid4_bioma_amazonia_era5_gee_2002-2012_0-25degree',
    fileFormat: 'CSV'
})
